In [ ]:
using Revise

In [ ]:
using Polyhedra
using CDDLib
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using StaticArrays
using RigidBodyTreeInspector
using MechanismComplementarity
using JuMP, Gurobi

In [ ]:
function DrakeVisualizer.addgeometry!(vis::Visualizer, obs::Obstacle, boundary::HRepresentation=SimpleHRepresentation(vcat(eye(3), -eye(3)), vcat([1., 0.5, 2.0], -[-1., -0.5, -0.1])))
    p = intersect(boundary, obs.interior)
    addgeometry!(vis, CDDPolyhedron{3, Float64}(p))
end

In [ ]:
urdf_mech = parse_urdf(Float64, "box_valkyrie.urdf")
mechanism, base = planar_revolute_base()
attach!(mechanism, base, urdf_mech)
world = root_body(mechanism)

basevis = Visualizer()[:boxval]
delete!(vis)
vis = basevis[:robot]
setgeometry!(vis, mechanism, parse_urdf("box_valkyrie.urdf", mechanism))
inspect(mechanism, vis)

In [ ]:
floor = planar_obstacle(default_frame(world), [0, 0, 1.], [0, 0, 0.])
wall = planar_obstacle(default_frame(world), [1., 0, 0], [-0.7, 0, 0])
free_space = space_between([floor, wall])

addgeometry!(basevis[:environment], wall)
addgeometry!(basevis[:environment], floor)

contact_limbs = findbody.(mechanism, ["rh", "lh", "rf", "lf"])
hands = findbody.(mechanism, ["rh", "lh"])
feet = findbody.(mechanism, ["rf", "lf"])

env = Environment(
    Dict(vcat(
            [body => ContactEnvironment(
                [Point3D(default_frame(body), SVector(0., 0, 0))],
                [floor],
                [free_space])
                for body in feet],
            [body => ContactEnvironment(
                [Point3D(default_frame(body), SVector(0., 0, 0))],
                [wall],
                [free_space])
                for body in hands])));

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "base_x"), [0])
set_configuration!(x0, findjoint(mechanism, "base_z"), [1.1])
set_velocity!(x0, findjoint(mechanism, "base_x"), [-1.0])
# set_velocity!(x0, findjoint(mechanism, "core_to_rf_x"), [-1])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-0.9])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-0.9])

In [ ]:
settransform!(vis, x0)

In [ ]:
controller = x -> begin
    kp = fill(10, num_velocities(x))
    kd = 0.1 * kp
    u = kp .* (configuration(x0) .- configuration(x)) .- kd .* velocity(x)
    u[1:3] .= 0
    u
end

Δt = 0.05
results = MechanismComplementarity.simulate(x0, controller, env, Δt, 100, GurobiSolver(OutputFlag=0));

In [ ]:
for r in results
    settransform!(vis, r.state)
    sleep(Δt)
end

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "base_x"), [0])
set_configuration!(x0, findjoint(mechanism, "base_z"), [1])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-1])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-1])

Δt = 0.05
input_limits = Dict([
        joint=>[-20, 10]
    for joint in keys(limits)])

model, results_opt = mc.optimize(x0, input_limits, limits, env, Δt, 8, Model(solver=GurobiSolver(TimeLimit=60)))
@objective model Min -100 * configuration(results_opt[end].state, findjoint(mechanism, "base_x"))[end] + sum([0.001 * sum(r.input.^2) for r in results_opt])

solve(model)
results_opt = getvalue.(results_opt);

In [ ]:
for r in results_opt
    settransform!(vis, r.state)
    sleep(Δt)
end

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "base_x"), [0])
set_configuration!(x0, findjoint(mechanism, "base_z"), [1])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-1])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-1])

Δt = 0.05
input_limits = Dict([
        joint=>[-20, 10]
    for joint in keys(limits)])

model, results_opt = mc.optimize(x0, input_limits, limits, env, Δt, 32, Model(solver=GurobiSolver(TimeLimit=120)))

objective = (
    -1 * configuration(results_opt[end].state, findjoint(mechanism, "base_x"))[end]
#     + 100 * velocity(results_opt[end].state, findjoint(mechanism, "base_x"))[end]^2
#     + 100 * velocity(results_opt[end].state, findjoint(mechanism, "base_z"))[end]^2
    )
# objective += sum([0.01 * sum(r.input.^2) for r in results_opt])

@objective model Min objective

for r in results_opt
    for (body, crs) in r.contacts
        for contact_result in crs
            JuMP.fix(contact_result.λ, 0)
        end
    end
    
    for (joint, jrs) in r.joint_contacts
        for joint_result in jrs
            JuMP.fix(joint_result.λ, 0)
        end
    end 
end

solve(model)
results_opt = getvalue.(results_opt);

In [ ]:
for r in results_opt
    settransform!(vis, r.state)
    sleep(Δt)
end

In [ ]:
i = 1
controller = x -> begin
    global i
    if i <= length(results_opt)
        i += 1
        results_opt[i - 1].input
    else
        kp = 20
        kd = 0.1 * kp
        u = kp .* (configuration(x0) .- configuration(x))[3:end] .- kd .* velocity(x)[3:end]
        vcat(zeros(2), u)
    end
        
end

results = mc.simulate(x0, controller, limits, env, Δt, 60, GurobiSolver(OutputFlag=0));

In [ ]:
settransform!(vis, results[1].state)

In [ ]:
for r in results
    settransform!(vis, r.state)
    sleep(Δt)
end